In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.agents import create_agent
import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

In [ ]:
## 掛け算と足し算を行うツールを定義
@tool
def get_multiple_choice_answer(a, b):
    """
    2つの数値を掛け合わせて結果を返します。
    """
    return a * b

@tool
def get_add(a, b):
    """
    2つの数値を足し合わせて結果を返します。
    """
    return a + b

def run_agent(input: str) -> str:
    tools = [get_multiple_choice_answer, get_add]
    systemprompt = "ユーザーからの質問に対して、必要に応じてツールを呼び出して答えてください。"
    llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=OPENAI_API_KEY)
    agent = create_agent(llm, tools, system_prompt=systemprompt)
    response = agent.invoke({"messages": [HumanMessage(content=input)]})
    return response["messages"][-1].content

In [4]:
run_agent("4と５を足してから８をかけて")

'4と5を足すと9になり、それに8をかけると72になります。'

In [13]:
## ニュースAPIを使用
url = "https://newsapi.org/v2/everything"
params = {
    
    "q":"サッカー",
    "sortBy":"publishedAt",
    "pazeSize":5,
    "apiKey":NEWS_API_KEY
}
requests.get(url,params).json()['articles'][0]['description']

'現在チャンピオンズリーグ（CL）の出場枠を「2」与えられているスコットランドだが、欧州カップ戦での不振を受けて、2026－27シーズンからCL出場枠が減らされることが決定的となったようだ。20日、イギリスメディア『BBC』が報じた。\u3000欧州サッカー連盟（UEFA）が主催するクラブ大会の各国出場枠は、過去5年間に出場したクラブの平均パフォーマンスから割り出す、UEFA係数ランキングによって決められ...'

In [39]:
# ニュースAPIを呼び出すツールを定義
@tool
def get_news(query:str) -> str:
    """
    最新のニュースを取得して、タイトルと説明を返します。
    """
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q':query,
        'sortBy':'publishedAt',
        'pazeSize':1,
        'apiKey':NEWS_API_KEY,
    }
    article = requests.get(url, params=params).json()['articles'][0]
    return f"タイトル: {article['title']} 説明:{article['description']}"

# ニュースを要約するツールを定義
@tool
def summarize(text:str) -> str:
    """
    ニュースの内容を要約して返します。
    """
    systemprompt = "ニュースの内容を要約してください。"
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3, api_key=OPENAI_API_KEY)
    responce = llm.invoke([SystemMessage(content=systemprompt),
                           HumanMessage(content=text)])
    return responce.content

# ニュースAPIを呼び出すツールと要約するツールをエージェントに追加
def run_agent(input:str) -> str:
    tools = [get_news, summarize]
    systemprompt = "ユーザーからの質問に対して、必要に応じてツールを呼び出して答えてください。"
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3, api_key=OPENAI_API_KEY)
    agent = create_agent(llm, tools, system_prompt=systemprompt)
    response = agent.invoke({"messages": [HumanMessage(content=input)]})
    return print(response["messages"][-1].content)

    
    

In [41]:
run_agent("サッカーの最新ニュースを要約して教えて")

スコットランドは現在、チャンピオンズリーグ（CL）の出場枠を2つ持っていますが、欧州カップ戦での成績不振により、2026－27シーズンからその出場枠が1つに減少する見込みです。これは、過去5年間のクラブの平均パフォーマンスに基づくUEFA係数ランキングによって決定されます。この変更は、セルティックなどのスコットランドのクラブにとって、リーグ優勝を逃すとCL出場が難しくなるため、大きな影響を及ぼす可能性があります。
